In [39]:
# reading training data
with open("input.txt","r") as file:
    text = file.read()

In [40]:
# length of the dataset
print("Length of the dataset in characters: ", len(text))

Length of the dataset in characters:  1115394


In [41]:
# inspect the data
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [42]:
# unique characters in the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Unique characters: {"".join(chars)}")
print(f"Number of unique characters: {vocab_size}")


Unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Number of unique characters: 65


In [43]:
# simple encoder decoder for our unique characters
char_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_char = {i:ch for i,ch in enumerate(chars)}

encode = lambda text: [char_to_int[ch] for ch in text]
decode = lambda int_list: "".join([int_to_char[i] for i in int_list])

In [44]:
print(encode("Hello"))


[20, 43, 50, 50, 53]


In [45]:
print(decode(encode("Hello")))

Hello


In [46]:
import torch
data =  torch.tensor(encode(text),dtype=torch.long)
print(data.shape)
print(data.dtype)
print(data[:500])

torch.Size([1115394])
torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [47]:
# data split into train and validation
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [48]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [49]:
x = train_data[:block_size]
print(x)
y= train_data[1:block_size+1]
print(y)
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target : {target}")

tensor([18, 47, 56, 57, 58,  1, 15, 47])
tensor([47, 56, 57, 58,  1, 15, 47, 58])
when input is tensor([18]) the target : 47
when input is tensor([18, 47]) the target : 56
when input is tensor([18, 47, 56]) the target : 57
when input is tensor([18, 47, 56, 57]) the target : 58
when input is tensor([18, 47, 56, 57, 58]) the target : 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target : 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target : 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target : 58


In [50]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print("inputs: ")
print(xb.shape)
print(xb)
print("targets: ")
print(yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context} the target : {target}")

inputs: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when input is tensor([24]) the target : 43
when input is tensor([24, 43]) the target : 58
when input is tensor([24, 43, 58]) the target : 5
when input is tensor([24, 43, 58,  5]) the target : 57
when input is tensor([24, 43, 58,  5, 57]) the target : 1
when input is tensor([24, 43, 58,  5, 57,  1]) the target : 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target : 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target : 39
when input is tensor([44]) the target : 53
when input is tensor([44, 53]) the target : 56
when input is tensor([44, 53, 56]) the target : 1
when input 

In [54]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    def forward(self, idx, targets):
        logits = self.token_embedding_table(idx)
        return logits
    
model = BigramLanguageModel(vocab_size)
out = model(xb,yb)
print(out.shape)


torch.Size([4, 8, 65])
